# Taxi Networks simulation

In this notebook, we will demonstrate code that simulates the behavior of taxis
in queueing networks. We are modeling this using a Jackson network, which models
taxis as customers traveling in the network. Taxis queue at each station $i$,
and are served by customers arriving at a rate of $\mu_i$. After a customer
gets on a taxi, it travels to another station $j$ with probability
$r_{ij}$. We assume the taxis 

In [1]:
import numpy as np
import scipy.linalg as la
import scipy.stats as sps
from matplotlib import pyplot as plt
from simulation import *
%matplotlib inline

In [2]:
test_nw = linear_network(5, 0.1, 1, 15)
print test_nw.graph[1].r
print test_nw.graph[4].r

{0: 0.22727272727272727, 2: 0.3181818181818182, 3: 0.22727272727272727, 4: 0.22727272727272727}
{0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25}


In [8]:
test_nw = linear_network(5, 0.1, 1, 15)
print [(nid, node.n) for nid, node in test_nw.graph.items()]
test_nw.tick()
print [(nid, node.n) for nid, node in test_nw.graph.items()]
test_nw.tick()
print [(nid, node.n) for nid, node in test_nw.graph.items()]
test_nw.tick()
print [(nid, node.n) for nid, node in test_nw.graph.items()]
test_nw.tick()
print [(nid, node.n) for nid, node in test_nw.graph.items()]


[(0, 15), (1, 15), (2, 15), (3, 15), (4, 15)]
[(0, 15), (1, 15), (2, 16), (3, 15), (4, 14)]
[(0, 14), (1, 15), (2, 16), (3, 15), (4, 15)]
[(0, 14), (1, 16), (2, 16), (3, 14), (4, 15)]
[(0, 13), (1, 16), (2, 16), (3, 14), (4, 16)]


In [3]:
from collections import Counter

test_nw = linear_network(10, 1, 1, 100)
counts = []
steps = 10000

for _ in range(steps):
    test_nw.tick()
    counts.append(test_nw.get_counts()[1])

x = range(steps)
for y in zip(*counts):
    plt.plot(x, y)

print map(np.mean, zip(*counts))

[9.2087000000000003, 28.801200000000001, 49.077500000000001, 128.8244, 77.412700000000001, 64.784499999999994, 99.701499999999996, 77.670000000000002, 84.765199999999993, 379.7543]


# Sandbox for math

In [5]:
def mva(v, mu, n):
    dim = len(v)
    gamma = v / mu
    L = np.zeros(dim)
    for k in range(1, n+1):
        L1 = np.ones(dim) + L
        f = k / float(gamma.dot(L1))
        L = f * gamma * L1
    return L

def get_first_evac(m):
    evals, evecs = la.eig(m, right=True)
    max_index = max(enumerate(evals), key=lambda x: abs(x[1]))[0]
    return evecs[:,max_index]

p_real = 1
p_virt = 0.0001
n = 5
m = linear_network(n, p_virt, p_real, 0).to_matrix()
mu = [(p_real + p_virt)]*(n-1) + [p_real]
print np.round(m, 5)

first_evec = get_first_evac(m)
first_evec /= first_evec[-1]

print first_evec / mu
first_evec = first_evec.astype(float)
print mva(first_evec, mu, n*30)

[[ 0.       0.24998  0.24998  0.24998  0.25   ]
 [ 0.25007  0.       0.24998  0.24998  0.25   ]
 [ 0.24998  0.25007  0.       0.24998  0.25   ]
 [ 0.24998  0.24998  0.25007  0.       0.25   ]
 [ 0.24998  0.24998  0.24998  0.25007  0.     ]]
[ 0.99984002+0.j  0.99992000+0.j  0.99992001+0.j  0.99992001+0.j
  1.00000000+0.j]
[ 29.93806828  29.99995281  29.99995777  29.99995777  30.06206337]


/usr/lib/python2.7/site-packages/IPython/kernel/__main__.py:27: ComplexWarning: Casting complex values to real discards the imaginary part


In [6]:
m = np.array([[0,0,0,0,0.25], [1,0,0,0,0.25], [0,1,0,0,0.25], [0,0,1,0,0.25], [0,0,0,1,0]])

first_evec = get_first_evac(m)
first_evec /= first_evec[-1]
print first_evec


[ 0.25+0.j  0.50+0.j  0.75+0.j  1.00+0.j  1.00+0.j]
